In [1]:
import tensorflow as tf
import gudhi as gd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.spatial as spatial
import json
from rich import print
from random import choice, sample
from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from gc import collect
from pickle import load

with open("9701_cech_persistence_entropy_012.pickle", mode = "rb") as data:
    save_dict = load(data)
    train_molecules = save_dict["train_molecules"]
    train_data = save_dict["train_data"]
    
    test_molecules = save_dict["test_molecules"]
    test_data = save_dict["test_data"]
    
%matplotlib inline
del save_dict
collect()

4

In [2]:
print(train_data.shape, test_data.shape)

(7761, 9)
(1940, 9)

# Extract descriptors
(this can be done from rdkit or from the descriptor .csv)

In [3]:
df = pd.read_csv("data/Drugbank_some_descriptors.csv")
cntr = 0
for d in df:
    cntr += 1
print(cntr)

126

### Filter descriptors which are unavailable

In [4]:
descriptors = list()

#---- consider only descriptors which have few missing values and which are floating point
for d in df:
    if df[d].isna().sum() < 800 and df[d].dtype == np.float64:
        descriptors.append(d)

#filter_descriptors = filter(lambda d : "OEselma Descriptors" not in d, descriptors)
#descriptors = list(set(descriptors) - set(filter_descriptors))
num_descriptors = len(descriptors)

#---- select only smiles which have these descriptor values
train_smiles = set(train_molecules)
test_smiles = set(test_molecules)

for i,d in enumerate(descriptors):
    avail_rows = df[np.logical_not(df[d].isna())]
    train_smiles = set(avail_rows["SMILES"]).intersection(train_smiles)
    test_smiles = set(avail_rows["SMILES"]).intersection(test_smiles)

train_idxs = np.asarray(sorted([ train_molecules.index(t) for t in train_smiles ]))
test_idxs = np.asarray(sorted([ test_molecules.index(t) for t in test_smiles ]))

train_molecules = np.asarray(train_molecules)
train_molecules = train_molecules[train_idxs]
train_data = train_data[train_idxs]

test_molecules = np.asarray(test_molecules)
test_molecules = test_molecules[test_idxs]
test_data = test_data[test_idxs]
#----
print(descriptors)

[
    'ACD Descriptors;logP',
    'ACD logD-logP;logP',
    'MOE Descriptors;h_ema',
    'MOE Descriptors;h_emd',
    'MOE Descriptors;h_emd_c',
    'MOE Descriptors;h_logp',
    'MOE Descriptors;h_logs',
    'MOE Descriptors;h_log_dbo',
    'MOE Descriptors;h_log_pbo',
    'MOE Descriptors;h_mr',
    'OEselma Descriptors;bromine_count',
    'OEselma Descriptors;carbon_count',
    'OEselma Descriptors;clorine_count',
    'OEselma Descriptors;ertl_tpsa',
    'OEselma Descriptors;fluorine_count',
    'OEselma Descriptors;hba',
    'OEselma Descriptors;hba_lipinski',
    'OEselma Descriptors;hbd',
    'OEselma Descriptors;hbd_lipinski',
    'OEselma Descriptors;halogen_count',
    'OEselma Descriptors;iodine_count',
    'OEselma Descriptors;max_flex_chain_1',
    'OEselma Descriptors;max_flex_chain_2',
    'OEselma Descriptors;max_flex_chain_3',
    'OEselma Descriptors;max_rigid_chain',
    'OEselma Descriptors;mol_volume_2d',
    'OEselma Descriptors;mw',
    'OEselma Descriptors;npsa',
    'OEselma Descriptors;neg_ioniz',
    'OEselma Descriptors;nitrogen_count',
    'OEselma Descriptors;nonpolar_count',
    'OEselma Descriptors;nonpolar_count_per_mw',
    'OEselma Descriptors;rigid_frag_count',
    'OEselma Descriptors;aromatic_ring_count',
    'OEselma Descriptors;atom_count',
    'OEselma Descriptors;bond_count',
    'OEselma Descriptors;rigid_bond_count',
    'OEselma Descriptors;ring_count',
    'OEselma Descriptors;rot_bond_count',
    'OEselma Descriptors;oxygen_count',
    'OEselma Descriptors;psa',
    'OEselma Descriptors;phosphorous_count',
    'OEselma Descriptors;polar_count',
    'OEselma Descriptors;polar_count_per_mw',
    'OEselma Descriptors;pos_ioniz',
    'OEselma Descriptors;silicon_count',
    'OEselma Descriptors;sulphur_count',
    'OEselma Descriptors;tsa',
    'Ertl TPSA;Ertl TPSA',
    'Exact Mass;Exact Mass',
    'Heavy Atom Count;Heavy Atom Count',
    'Molecular Volume (2D);Molecular Volume (2D)',
    'Molecular Weight;Molecular Weight',
    'NPSA;NPSA',
    'PSA;PSA',
    'Ring Count;Ring Count',
    'Rotatable Bond Count;Rotatable Bond Count',
    'AZlogD74;prediction',
    'AZlogD74;sd_prediction',
    'ChromlogD;prediction',
    'ChromlogD;sd_prediction',
    'ALogP',
    'Mol Weight',
    'Molar Refractivity',
    'Polarizability',
    'TPSA',
    'Solubility DD Class;confidence',
    'Solubility DD Class;credibility',
    'SCScore;SCScore',
    'AZlogD74 (NN);nearest_neighbours[]- GT similarity',
    'AZlogD74 (NN);nearest_neighbours[]- GT measured',
    'clogP;clogp',
    'clogP;clogp_error',
    'OEselma Descriptors;part_flex_chain'
]

In [5]:
print(train_data.shape, test_data.shape)

(7205, 9)
(1787, 9)

### Build numpy arrays of descriptors

In [6]:
collect()
train_labels = np.empty((train_data.shape[0], num_descriptors))
test_labels = np.empty((test_data.shape[0], num_descriptors))
train_labels[:] = np.nan
test_labels[:] = np.nan

for i,mol in enumerate(tqdm(train_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    train_labels[i,:] = np.asarray(data_vec)

for i,mol in enumerate(tqdm(test_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    test_labels[i,:] = np.asarray(data_vec)

  0%|          | 0/7205 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

In [7]:
print(train_labels.shape, test_labels.shape)

(7205, 74)
(1787, 74)

In [8]:
print(not np.isnan(train_labels).any(), not np.isnan(test_labels).any())

True True

In [9]:
with np.printoptions(2, suppress = True):
    print(choice(train_labels))

[  0.34   0.34   7.32   9.07   8.     5.06   7.61  -2.66   1.29   0.
   0.    13.     0.    82.05   0.     7.     7.     1.     1.     0.
   0.     5.     0.     0.     9.   475.04 280.3  264.99   0.     4.
  10.     0.04   4.     2.    20.    21.    12.     2.     5.     3.
  71.06   0.     3.     0.01   0.     0.     0.   336.    82.05 280.15
  20.   475.04 280.3  264.99  71.06   2.     5.     0.51   0.15   0.51
   0.3    0.27 280.32  74.15  29.06  78.67   0.91   0.35   4.21   0.78
  -0.49   0.4   10.     5.  ]

# Setup OLS model

In [10]:
from sklearn import linear_model

#creating a regression object
reg = linear_model.LinearRegression()

#runnin OLS on your data, assuming that you already have arrays x and y
reg.fit( train_data, train_labels )
reg.score(train_data, train_labels)

0.21731159396226127

In [11]:
reg.coef_.shape

(74, 9)

In [12]:
# Using 'auto'/'sum_over_batch_size' reduction type.
mae = tf.keras.losses.MeanAbsoluteError()
mae(test_labels, reg.predict(test_data)).numpy()

2022-01-13 19:55:02.725887: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-13 19:55:03.832370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6814 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


19.298789978027344

In [13]:
predicted = reg.predict(test_data)

In [14]:
out_dict = dict()
out_dict["Descriptor"] = list()
out_dict["_sigma"] = list()
out_dict["Average"] = list()
out_dict["Test Average"] = test_labels.mean(axis = 0)
out_dict["MAE"] = np.abs(test_labels - predicted).mean(axis = 0)

for i,d in enumerate(descriptors):
    out_dict["Descriptor"].append(d)
    out_dict["Average"].append(df[d].mean(skipna = True)) # population mean for this descriptor
    out_dict["_sigma"].append(df[d].std(skipna = True)) # population mean for this descriptor
    
out_dict["MAE/sigma"] = out_dict["MAE"]/out_dict["_sigma"]

pd.set_option("display.max_rows", None, "display.max_columns", None)
out_df = pd.DataFrame(data = out_dict)
print(out_dict["MAE"].sum()/len(descriptors)) # THIS IS WHAT tf.keras.losses.MeanAbsoluteError() DOES
out_df = out_df.sort_values("MAE/sigma")
out_df

19.298790657527803

,Descriptor,_sigma,Average,Test Average,MAE,MAE/sigma
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.002336,0.021538
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.237878
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.067484,0.254362
62,Mol Weight,301.265463,374.512520,371.636732,102.311704,0.339606
64,Polarizability,29.516618,37.783778,37.861987,10.196873,0.345462
63,Molar Refractivity,74.060489,95.559089,95.940680,25.766911,0.347917
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.458272,0.356848
65,TPSA,133.333536,107.810334,104.397905,50.274115,0.377055
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,104.243409,0.398349
56,Rotatable Bond Count;Rotatable Bond Count,7.213711,6.681677,6.670397,2.915731,0.404193


In [15]:
latex_dict = dict()
latex_dict["Descriptors"]                              = out_dict["Descriptor"]
latex_dict["$\sigma$"]                                 = out_dict["_sigma"]
latex_dict["$\mu$"]                                    = out_dict["Average"]
latex_dict["$\overline{\mathbf{f}_d}$"]                = out_dict["Test Average"]
latex_dict["$\overline{|\mathbf{y}_d-\mathbf{f}_d|}$"] = out_dict["MAE"] 
latex_dict["Score"]                                    = out_dict["MAE/sigma"]
latex_df = pd.DataFrame(data = latex_dict)
latex_df = latex_df.sort_values("Score")
latex_df

,Descriptors,$\sigma$,$\mu$,$\overline{\mathbf{f}_d}$,$\overline{|\mathbf{y}_d-\mathbf{f}_d|}$,Score
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.002336,0.021538
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.237878
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.067484,0.254362
62,Mol Weight,301.265463,374.512520,371.636732,102.311704,0.339606
64,Polarizability,29.516618,37.783778,37.861987,10.196873,0.345462
63,Molar Refractivity,74.060489,95.559089,95.940680,25.766911,0.347917
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.458272,0.356848
65,TPSA,133.333536,107.810334,104.397905,50.274115,0.377055
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,104.243409,0.398349
56,Rotatable Bond Count;Rotatable Bond Count,7.213711,6.681677,6.670397,2.915731,0.404193


In [16]:
latex_string = latex_df.to_latex(index = False, escape = False)
latex_string = latex_string.replace("_", " ")
latex_string = latex_string.replace("OEselma Descriptors;", "OEselma ")
latex_string = latex_string.replace("&\n"," & ")
latex_string = latex_string.replace("\n&"," & ")
latex_string = latex_string.replace("& \n"," & ")
latex_string = latex_string.replace("MOE Descriptors;", "MOE ")

lines = latex_string.split("\n")
for i,l in enumerate(lines):
    if i < 4 or i > len(lines)-4:
        continue
    
    a = l.split("&")
    b = a[1:]
    c = str.title(a[0].split(";")[0])
    d = [ c ] + b
    lines[i] = "&".join(d)
latex_string = "\n".join(lines)

latex_string = latex_string.replace("&", " & ")

for i in range(40):
    latex_string = latex_string.replace("  ", " ")


print(latex_string)

\begin{tabular}{lrrrrr}
\toprule
 Descriptors & $\sigma$ & $\mu$ & $\overline{\mathbf{f} d}$ & $\overline{|\mathbf{y} 
d-\mathbf{f} d|}$ & Score \\
\midrule
 Oeselma Silicon Count & 0.108439 & 0.004239 & 0.000560 & 0.002336 & 0.021538 \\
 Oeselma Iodine Count & 0.287843 & 0.028450 & 0.039731 & 0.068471 & 0.237878 \\
 Oeselma Bromine Count & 0.265308 & 0.034856 & 0.033016 & 0.067484 & 0.254362 \\
 Mol Weight & 301.265463 & 374.512520 & 371.636732 & 102.311704 & 0.339606 \\
 Polarizability & 29.516618 & 37.783778 & 37.861987 & 10.196873 & 0.345462 \\
 Molar Refractivity & 74.060489 & 95.559089 & 95.940680 & 25.766911 & 0.347917 \\
 Moe H Ema & 9.691163 & 8.864515 & 8.293012 & 3.458272 & 0.356848 \\
 Tpsa & 133.333536 & 107.810334 & 104.397905 & 50.274115 & 0.377055 \\
 Oeselma Tsa & 261.688962 & 414.138860 & 413.916620 & 104.243409 & 0.398349 \\
 Rotatable Bond Count & 7.213711 & 6.681677 & 6.670397 & 2.915731 & 0.404193 \\
 Oeselma Rot Bond Count & 7.213711 & 6.681677 & 6.670397 & 2.915731 & 0.404193 \\
 Molecular Volume (2D) & 371.986946 & 592.330629 & 587.763424 & 155.383329 & 0.417712 \\
 Oeselma Mol Volume 2D & 371.986946 & 592.330629 & 587.763424 & 155.383329 & 0.417712 \\
 Moe H Logs & 5.975325 & 9.838051 & 9.752189 & 2.600139 & 0.435146 \\
 Oeselma Atom Count & 15.766603 & 25.429675 & 25.508674 & 6.920609 & 0.438941 \\
 Heavy Atom Count & 15.757548 & 25.426522 & 25.506995 & 6.919384 & 0.439116 \\
 Oeselma Mw & 227.821937 & 373.328639 & 371.636766 & 102.316792 & 0.449109 \\
 Molecular Weight & 227.821937 & 373.328639 & 371.636766 & 102.316792 & 0.449109 \\
 Exact Mass & 227.548842 & 372.828687 & 371.268124 & 102.207349 & 0.449167 \\
 Oeselma Bond Count & 16.980448 & 26.935092 & 27.180750 & 7.750208 & 0.456420 \\
 Npsa & 200.207940 & 307.642466 & 311.829856 & 91.812098 & 0.458584 \\
 Oeselma Npsa & 200.207940 & 307.642466 & 311.829856 & 91.812098 & 0.458584 \\
 Oeselma Phosphorous Count & 0.474829 & 0.122751 & 0.123111 & 0.221552 & 0.466593 \\
 Moe H Emd & 10.821464 & 12.580101 & 12.460378 & 5.054848 & 0.467113 \\
 Oeselma Carbon Count & 11.377305 & 17.636929 & 17.901511 & 5.320611 & 0.467651 \\
 Moe H Emd C & 5.020204 & 6.492526 & 6.514657 & 2.400257 & 0.478119 \\
 Psa & 101.197251 & 106.494461 & 102.077127 & 50.681520 & 0.500819 \\
 Oeselma Psa & 101.197251 & 106.494461 & 102.077127 & 50.681520 & 0.500819 \\
 Oeselma Part Flex Chain & 4.183248 & 4.829146 & 4.982652 & 2.105559 & 0.503331 \\
 Ertl Tpsa & 94.357347 & 102.704204 & 99.096475 & 47.579564 & 0.504249 \\
 Oeselma Ertl Tpsa & 94.357347 & 102.704204 & 99.096475 & 47.579564 & 0.504249 \\
 Oeselma Polar Count & 5.731492 & 6.059067 & 5.834359 & 2.927670 & 0.510804 \\
 Oeselma Hba Lipinski & 5.725688 & 6.789826 & 6.635702 & 2.928607 & 0.511486 \\
 Oeselma Fluorine Count & 0.933018 & 0.276967 & 0.304421 & 0.500981 & 0.536947 \\
 Oeselma Rigid Frag Count & 3.579486 & 5.023363 & 4.985450 & 1.952665 & 0.545516 \\
 Oeselma Hbd Lipinski & 3.595447 & 2.940744 & 2.945719 & 1.962018 & 0.545695 \\
 Oeselma Hbd & 3.482418 & 2.674517 & 2.675993 & 1.900881 & 0.545851 \\
 Moe H Log Pbo & 3.788166 & 1.932621 & 2.141476 & 2.082669 & 0.549783 \\
 Oeselma Oxygen Count & 4.226225 & 4.206029 & 4.003358 & 2.338538 & 0.553340 \\
 Oeselma Hba & 4.799591 & 5.812529 & 5.787913 & 2.668917 & 0.556072 \\
 Azlogd74 & 0.652538 & 0.729417 & 0.712565 & 0.371994 & 0.570072 \\
 Oeselma Max Flex Chain 2 & 1.948082 & 1.691916 & 1.711807 & 1.115499 & 0.572614 \\
 Oeselma Max Flex Chain 1 & 2.645862 & 2.943565 & 3.026861 & 1.517344 & 0.573478 \\
 Oeselma Max Flex Chain 3 & 1.623217 & 1.148954 & 1.130946 & 0.931127 & 0.573630 \\
 Oeselma Neg Ioniz & 1.108872 & 0.539802 & 0.452714 & 0.641812 & 0.578798 \\
 Oeselma Clorine Count & 0.515279 & 0.168629 & 0.182988 & 0.301226 & 0.584589 \\
 Moe H Log Dbo & 3.066570 & -3.836500 & -3.840843 & 1.803478 & 0.588109 \\
 Oeselma Nitrogen Count & 2.861234 & 2.583797 & 2.632345 & 1.690824 & 0.590942 \\
 Oeselma Rigid Bond Count & 10.405612 & 16.273575 & 16.537213 & 6.2255